In [18]:
from __future__ import division
from __future__ import print_function

import sys
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from DL import DataLoader, Batch
import decode_beam
import tensorflow as tf
from SamplePreprocessor import preprocess
import os
import pathlib
import BestPath
import Common
import time
import copy
import math


In [19]:
class DecoderType:
    BestPath = 0
    BeamSearch = 1
    WordBeamSearch = 2

In [20]:
class FilePaths:
    "filenames and paths to data"
    fnCharList = (r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\model\charList.txt')
    fnAccuracy = (r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\model\accuracy.txt')
    #c = (r'C:\Users')
    #fnTrain = os.path.join(c, "User", "fastai", "courses", "dl1","Handwriting recognition project","data")
    #fnTrain = (r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\data')
    fnTrain = pathlib.Path(r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\data')
    #print (fnTrain)
    fnInfer = (r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\data\test.png')
    fnCorpus = (r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\data\corpus.txt')


In [21]:
class Model(torch.nn.Module): 
        
    # model constants
    batchSize = 50
    imgSize = (128, 32)
    maxTextLen = 32
    
    def __init__(self):
        super(Model, self).__init__()
        self.snapID = 0
        #self.is_train = tf.placeholder(tf.bool, name='is_train')
        
        # input image batch
        #self.inputImgs = tf.placeholder(tf.float32, shape=(None, Model.imgSize[0], Model.imgSize[1]))
        #self.inputImgs = torch.tensor(batch.imgs, Model.imgSize[0], Model.imgSize[1])
        
        #cnn function
        self.conv1 = torch.nn.Conv2d(1, 32, 5, stride = 1, padding = 2).cuda()
        self.batchnorm1 = torch.nn.BatchNorm2d(32).cuda().cuda()
        self.relu1 = torch.nn.ReLU().cuda()
        self.pool1 = torch.nn.MaxPool2d((2,2), stride = (2,2)).cuda()
        
        self.conv2 = torch.nn.Conv2d(32, 64, 5, stride = 1, padding = 2).cuda()
        self.batchnorm2 = torch.nn.BatchNorm2d(64).cuda()
        self.relu2 = torch.nn.ReLU().cuda()
        self.pool2 = torch.nn.MaxPool2d((2,2), stride = (2,2)).cuda()
        
        self.conv3 = torch.nn.Conv2d(64, 128, 3, stride = 1, padding = 1).cuda()
        self.batchnorm3 = torch.nn.BatchNorm2d(128).cuda()
        self.relu3 = torch.nn.ReLU().cuda()
        self.pool3 = torch.nn.MaxPool2d((1,2), stride = (1,2)).cuda()  
        
        self.conv4 = torch.nn.Conv2d(128, 128, 3, stride = 1, padding = 1).cuda()
        self.batchnorm4 = torch.nn.BatchNorm2d(128).cuda()
        self.relu4 = torch.nn.ReLU().cuda()
        self.pool4 = torch.nn.MaxPool2d((1,2), stride = (1,2)).cuda()
        
        self.conv5 = torch.nn.Conv2d(128, 256, 3, stride = 1, padding = 1).cuda()
        self.batchnorm5 = torch.nn.BatchNorm2d(256).cuda()
        self.relu5 = torch.nn.ReLU().cuda()
        self.pool5 = torch.nn.MaxPool2d((1,2), stride = (1,2)).cuda()
        
        #rnn function
        self.lstm = torch.nn.LSTM(256, 256, 2, batch_first = True, bidirectional = True).cuda()
        # BxTxH + BxTxH -> BxTx2H -> BxTx1x2H 
        #squeeze
        self.rnnconv2d = torch.nn.Conv2d(512, 80, 1, stride = 1, padding = 0).cuda()
        
    def forward(self, inputImgs):
        #cnn forward pass
        inputTensor = torch.from_numpy(inputImgs).cuda()
        inputTensor = inputTensor.type(torch.FloatTensor).cuda()
        inputTensor = torch.unsqueeze(inputTensor, 1)
        #print (inputTensor.size()) [50,1,128,32]
        out = self.conv1(inputTensor)
        out = self.batchnorm1(out)
        out = self.relu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.batchnorm2(out)
        out = self.relu2(out)
        out = self.pool2(out)
        out = self.conv3(out)
        out = self.batchnorm3(out)
        out = self.relu3(out)
        out = self.pool3(out)
        out = self.conv4(out)
        out = self.batchnorm4(out)
        out = self.relu4(out)
        out = self.pool4(out)
        out = self.conv5(out)
        out = self.batchnorm5(out)
        out = self.relu5(out)
        out = self.pool5(out)
        
        #rnn forward pass
        #print (cnn.size())
        out = torch.squeeze(out, 3)
        out = out.permute(0,2,1)
        #print (cnn.size()) cnn= [50,32,256]
        #h0, c0 shape (num_layers * num_directions, batch, hidden_size):
        h0 = torch.zeros(4, out.size(0), 256).cuda()
        c0 = torch.zeros(4, out.size(0), 256).cuda()
        #packed_cnn = torch.nn.utils.rnn.pack_padded_sequence(cnn, [32]*cnn.size(1))
        #print(packed_cnn.size())
        out, _ = self.lstm(out, (h0,c0))
        #print (rnn_out.size())
        #rnn_out, _ = torch.nn.utils.rnn.pad_packed_sequence(rnn_out, batch_first=False)
        #print (rnn_out.size()) [50,32,512]
        out = torch.unsqueeze(out, 3) #[50,512,32,1]
        out = out.permute(0,2,1,3) #[50,32,1,512]
        #print (rnn_out.size())
        #print (rnn_out.size())
        out = self.rnnconv2d(out)
        #out = self.fnl(rnn_out)
        return out

In [22]:
#literally just copy this function
def toSparse(texts):
    "put ground truth texts into sparse tensor for ctc_loss"
    indices = []
    values = []
    shape = [len(texts), 0] # last entry must be max(labelList[i])
    charList = open(FilePaths.fnCharList).read()
    # go over all texts
    for (batchElement, text) in enumerate(texts):
        # convert to string of label (i.e. class-ids)
        labelStr = [charList.index(c) for c in text]
        # sparse tensor must have size of max. label-string
        if len(labelStr) > shape[1]:
            shape[1] = len(labelStr)
        # put each label into sparse tensor
        for (i, label) in enumerate(labelStr):
            indices.append([batchElement, i])
            values.append(label)

    return (indices, values, shape)

In [23]:
def fromSparse(indices, values, shape):
    "put ground truth texts into sparse tensor for ctc_loss"
    outputMatrix = torch.zeros(shape[0], shape[1], dtype = torch.long).cuda()
    numValues = len(values)
    targetLen = [0]*shape[0]
    index = 0
    for i in range(numValues):
        if indices[i][0] != 0 and indices[i][0] != indices[i-1][0]:
            index +=1
        dim0 = indices[i][0]
        dim1 = indices[i][1]
        outputMatrix[dim0, dim1]= values[i]
        targetLen[index]+=1
    return outputMatrix, targetLen

In [24]:
def recLabelingProb(t, s, mat, labelingWithBlanks, blank, cache):
	"recursively compute probability of labeling, save results of sub-problems in cache to avoid recalculating them"

	# check index of labeling
	if s < 0:
		return 0.0

	# sub-problem already computed
	if cache[t][s] != None:
		return cache[t][s]

	# initial values
	if t == 0:
		if s == 0:
			res = mat[0, blank]
		elif s == 1:
			res = mat[0, labelingWithBlanks[1]]
		else:
			res = 0.0

		cache[t][s] = res
		return res

	# recursion on s and t
	res = (recLabelingProb(t-1, s, mat, labelingWithBlanks, blank, cache) + recLabelingProb(t-1, s-1, mat, labelingWithBlanks, blank, cache)) * mat[t, labelingWithBlanks[s]]

	# in case of a blank or a repeated label, we only consider s and s-1 at t-1, so we're done
	if labelingWithBlanks[s] == blank or (s >= 2 and labelingWithBlanks[s-2] == labelingWithBlanks[s]):
		cache[t][s] = res
		return res

	# otherwise, in case of a non-blank and non-repeated label, we additionally add s-2 at t-1
	res += recLabelingProb(t-1, s-2, mat, labelingWithBlanks, blank, cache) * mat[t, labelingWithBlanks[s]]
	cache[t][s] = res
	return res

In [25]:
def emptyCache(maxT, labelingWithBlanks):
	"create empty cache"
	return [[None for _ in range(len(labelingWithBlanks))] for _ in range(maxT)]

In [26]:
def ctcLabelingProb(mat, gt, classes):
	"calculate probability p(gt|mat) of a given labeling gt and a matrix mat according to section 'The CTC Forward-Backward Algorithm' in Graves paper"
	maxT, _ = mat.shape # size of input matrix
	blank = len(classes) # index of blank label
	labelingWithBlanks = Common.extendByBlanks(Common.wordToLabelSeq(gt, classes), blank) # ground truth text as label string extended by blanks
	cache = emptyCache(maxT, labelingWithBlanks) # cache subresults to avoid recalculating  subproblems over and over again
	return recLabelingProb(maxT-1, len(labelingWithBlanks)-1, mat, labelingWithBlanks, blank, cache) + recLabelingProb(maxT-1, len(labelingWithBlanks)-2, mat, labelingWithBlanks, blank, cache)

In [27]:
def softmax(mat):
    "calc softmax such that labels per time-step form probability distribution"
    maxT, _ = mat.shape # dim0=t, dim1=c
    res = np.zeros(mat.shape)
    for t in range(maxT):
        y = mat[t, :]
        e = np.exp(y)
        s = np.sum(e)
        if math.isinf(s):
            res[t, :] = 0
        else:
            res[t, :] = e/s
    return res

In [28]:
def printStats(phase, loss, acc, prob, best_prob):
    string = "{} Loss: {:.4f} Epoch Acc: {:.2E} Epoch Prob: {:.2E} Best Probability: {:.2E}".format(phase, loss, acc, prob, best_prob)
    return string

In [29]:
def printEpoch(epoch, num_epoch, start):
    string = "Epoch {}/{} : {} - {:.0f}h {:.0f}m\n{}\n".format(epoch, num_epoch, time.asctime(),
                                                             (time.time() - start)//3600, ((time.time() - start)//60)%60, '-' * 10)
    return string

In [30]:
def getdata(phase):
    loader = DataLoader(filePath = FilePaths.fnTrain, batchSize = Model.batchSize, imgSize = Model.imgSize, maxTextLen = Model.maxTextLen)
    if phase == 'train':
        loader.trainSet()
    else:
        loader.validationSet()
    batch, labels, wlabels = [], [], []
    batchNumber = 0
    while loader.hasNext():
#        iterInfo = loader.getIteratorInfo()
        batch.append(loader.getNext())
        labels.append(toSparse(batch[batchNumber].gtTexts))
        wlabels.append(batch[batchNumber].gtTexts)
        numBatchElements = len(batch[batchNumber].imgs)
#        seqLen.append(Model.maxTextLen * numBatchElements)
        batchNumber+= 1
    numberOfBatches = batchNumber
    charList = loader.charList
    return batch, labels, wlabels, numberOfBatches, charList 

In [39]:
def train_model(model, num_epochs=25, learning_rate = 0.01, decoderType=DecoderType.BestPath):
    since = time.time()
    ctcloss, optimizer = createLossAndOptimizer(model, learning_rate)
    # prepare information about language (dictionary, characters in dataset, characters forming words)    
    wordChars = open(r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\model\charList.txt').read().splitlines()[0]
    corpus = open(r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\data\corpus.txt').read()
    PATH = (r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\model\checkpoint.pt')
    #get input and labels
    batchtrain, labelstrain, wlabelstrain, numberOfBatchestrain, charList = getdata('train')
    batchval, labelsval, wlabelsval, numberOfBatchesval, charList = getdata('val')
    chars = str().join(charList)
    best_prob = 0.0
    # Loading checkpoint
    if os.path.exists(PATH):
        print("Checkpoint loaded")
        checkpoint = torch.load(PATH)
        model.state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        current_epoch = checkpoint['epoch'] + 1
        best_prob = checkpoint['best_prob']
        best_model_wts = checkpoint['best_wts']
    else:
        best_prob = 0.0
        current_epoch = 0
        best_model_wts = copy.deepcopy(model.state_dict())
        
    for epoch in range(current_epoch, num_epochs):
        if epoch < 100:
            learning_rate = 0.01
        elif epoch < 10000: 
            learning_rate = 0.001
        else:
            learning_rate = 0.0001
            
        print(printEpoch(epoch, num_epochs - 1, since))
        #compute output and loss
        for phase in ['train','val']:
            if phase == 'train':
                model.train()
                batch, labels, wlabels, numberOfBatches, charList = batchtrain, labelstrain, wlabelstrain, numberOfBatchestrain, charList
            else:
                model.eval()
                batch, labels, wlabels, numberOfBatches, charList = batchval, labelsval, wlabelsval, numberOfBatchesval, charList
            
            running_loss = 0.0
            running_accuracy = 0.0
            running_probability = 0.0
            # zero the parameter gradients    
            model.zero_grad()
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                torch.autograd.set_detect_anomaly(True)
                for batchNumber in range(numberOfBatches):
                    optimizer.zero_grad()
                    #print (batch[batchNumber].imgs.shape)
                    out= model(batch[batchNumber].imgs)
                    #print (out.size())
                    out = out.squeeze(3)
                    out = out.permute(2,0,1)
                    #out size [32,50,80]
                    
                    #print (out)
                    #print (type(out))
                    #print (out.size())
                    #print (len(wlabels[batchNumber]))
                    #print (wlabels[batchNumber])
                    #labels[batchNumber] include indices, values (based on charlist.txt), shapes(batchsize, maxtextlength in batch)
                    #print (len(labels[batchNumber]))
                    #print (labels[batchNumber])
                    targets, targetsLen = fromSparse(labels[batchNumber][0], labels[batchNumber][1], labels[batchNumber][2])
                    #targets is a matrix [batchsize, maxtextlength] with value in index correspond with charList
                    #print (type(targets[0,0]))
                    #print (targets[0,0])
                    #inputsLen = tuple([32] * labels[batchNumber][2][0])
                    inputsLen = torch.full(size=(50,), fill_value=32, dtype=torch.long).cuda()
                    targetsLen = torch.LongTensor(targetsLen).cuda()
                    #print (targets)
                    #print (targetsLen)
                    # calc loss for batch
                    
                    logProbs = torch.nn.functional.log_softmax(out, dim = 2).requires_grad_().cuda()
                    #print (logProbs)
                    loss = torch.mean(ctcloss(logProbs, targets, inputsLen,targetsLen)).requires_grad_().cuda()
                    #loss = ctcloss(logProbs, targets, inputsLen,targetsLen).requires_grad_().cuda()
                    #print (logProbs)
                    
                    #out2 = out.permute(1,0,2)
                    out2 = out.permute(1,0,2)
                    #print(out2.is_cuda)
                    #out2 = out2.cpu().detach().numpy()
                    outputText = []
                    accurateWords = 0
                    prob = []
                    #print(loss.is_cuda)
                    
                    for i in range (out2.shape[0]):
                        #print (wlabels[batchNumber][i])
                        #print (prob[i])
                        #decode output to Text
                        #print (softmaxOut)
                        softmaxOut = torch.nn.functional.softmax(out2[i], dim = 1).cuda()
                        softmaxOut = softmaxOut.cpu().detach().numpy()
                        #print (softmaxOut)
                        outputText.append(BestPath.ctcBestPath(softmaxOut, charList))
                        #calculate confidence of each words
                        prob.append(ctcLabelingProb(softmaxOut, wlabels[batchNumber][i],charList))
                        #print (prob[i])
                        #if math.isnan(prob[i]):
                            #print (prob[i])
                            #print (wlabels[batchNumber][i])                            
                            #print (batchNumber)
                            #print (i)
                        if outputText[i] == wlabels[batchNumber][i]:
                            accurateWords +=1
                    batchAccuracy = accurateWords/len(outputText)
                    #print (wlabels[batchNumber])
                    batchProb = sum(prob)/len(prob)
                    #print (outputText)
                    #print (batchProb)
#                    print (len(outputText))
#                     tfOut = out.detach().numpy()
#                     print (tfOut)
#                     print (type(tfOut))
#                     sess = tf.Session()
#                     with sess.as_default():
#                         tfOut = tf.constant(tfOut, dtype = tf.float32)
#                     print (tfOut)
#                     print (type(tfOut))
#                     # decoder: either best path decoding or beam search decoding
#                     if decoderType == DecoderType.BestPath:
#                         decoder = tf.nn.ctc_greedy_decoder(inputs=tfOut, sequence_length=seqLen[batchNumber])
#                     elif decoderType == DecoderType.BeamSearch:
#                         decoder = tf.nn.ctc_beam_search_decoder(inputs=out, sequence_length=seqLen[batchNumber], beam_width=50, merge_repeated=False)
#                     elif self.decoderType == DecoderType.WordBeamSearch:
#                         # import compiled word beam search operation (see https://github.com/githubharald/CTCWordBeamSearch)
#                         word_beam_search_module = tf.load_op_library(r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\CTCWordBeamSearch-master\cpp\proj\TFWordBeamSearch.so')
#                         # decode using the "Words" mode of word beam search
#                         decoder = word_beam_search_module.word_beam_search(tf.nn.softmax(out, dim=2), 50, 'Words', 0.0, corpus.encode('utf8'), chars.encode('utf8'), wordChars.encode('utf8'))
                        
                    
                #decoderOuputToText function                   
                    "extract texts from output of CTC decoder"
                    # contains string of labels for each batch element
#                    batchSize = len(batch[batchNumber].imgs)
#                    encodedLabelStrs = [[] for i in range(batchSize)]

#                     # word beam search: label strings terminated by blank
#                     if decoderType == DecoderType.WordBeamSearch:
#                         blank=len(charList)
#                         for b in range(batchSize):
#                             #not sure what ctcOutput need to look at infer batch function in model
#                             for label in ctcOutput[b]:
#                                 if label==blank:
#                                     break
#                                 encodedLabelStrs[b].append(label)

#                     # TF decoders: label strings are contained in sparse tensor
#                     else:
#                         # ctc returns tuple, first element is SparseTensor 
#                         decoded=ctcOutput[0][0] 

#                         # go over all indices and save mapping: batch -> values
#                         idxDict = { b : [] for b in range(batchSize) }
#                         for (idx, idx2d) in enumerate(decoded.indices):
#                             label = decoded.values[idx]
#                             batchElement = idx2d[0] # index according to [b,t]
#                             encodedLabelStrs[batchElement].append(label)

                    # map labels to chars for all batch elements
#                    texts = [str().join([self.charList[c] for c in labelStr]) for labelStr in encodedLabelStrs]

                    
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    running_loss += loss.item()
                    running_accuracy += batchAccuracy
                    running_probability += batchProb
                    #print (type(running_corrects))
                    #we might not have an accuracy rate since it's text
                    #running_corrects += torch.sum(preds == labs)#.cuda()
                    
            epoch_loss = running_loss / numberOfBatches
            epoch_acc = running_accuracy / numberOfBatches
            epoch_prob = running_probability / numberOfBatches
            if phase == 'val' and epoch == 0:
                test_batch = batch[0].imgs
                test_labels = wlabels[0]
            #if phase == 'val' and epoch_acc > best_acc:
            if phase == 'val' and epoch_prob > best_prob:
                best_prob = epoch_prob
                best_model_wts = copy.deepcopy(model.state_dict())
                model.load_state_dict(best_model_wts)
                torch.save({
                    'model': model,
                    'test_batch': test_batch,
                    'test_labels': test_labels
                }, r'C:\Users\User\fastai\courses\dl1\Handwriting recognition project\model\model.pt')
                print("Model saved")
            print(printStats(phase, epoch_loss, epoch_acc, epoch_prob, best_prob))
        if (epoch + 1) % 5 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_prob': best_prob,
                'best_wts': best_model_wts
            }, PATH)
            print("Checkpoint saved")
            
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}hr {:.0f}m {:.0f}s'.format(
        time_elapsed//3600, (time_elapsed // 60) % 60, time_elapsed % 60))
    #use the latest model since no improvement in accuracy yet
    model.load_state_dict(best_model_wts)
    
    return model, test_batch, test_labels

In [40]:
def createLossAndOptimizer(net, learning_rate = 0.01):
    ctcloss = torch.nn.CTCLoss(reduction ='none').cuda()
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, weight_decay=0.00005)
    return ctcloss, optimizer
        
        
        

In [41]:
model_ft, test_batch, test_labels = train_model(Model().cuda(), num_epochs = 400, learning_rate = 0.01)

Epoch 0/399 : Mon May  6 19:36:17 2019 - 0h 1m
----------

train Loss: 14.2759 Epoch Acc: 0.00E+00 Epoch Prob: 3.56E-13 Best Probability: 0.00E+00


D:\Anaconda\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model saved
val Loss: 11.9318 Epoch Acc: 0.00E+00 Epoch Prob: 7.39E-49 Best Probability: 7.39E-49
Epoch 1/399 : Mon May  6 19:40:15 2019 - 0h 5m
----------

train Loss: 12.0768 Epoch Acc: 0.00E+00 Epoch Prob: 1.03E-11 Best Probability: 7.39E-49
Model saved
val Loss: 10.0745 Epoch Acc: 0.00E+00 Epoch Prob: 1.62E-41 Best Probability: 1.62E-41
Epoch 2/399 : Mon May  6 19:44:12 2019 - 0h 9m
----------

train Loss: 9.9977 Epoch Acc: 0.00E+00 Epoch Prob: 8.70E-11 Best Probability: 1.62E-41
Model saved
val Loss: 9.0228 Epoch Acc: 0.00E+00 Epoch Prob: 1.89E-32 Best Probability: 1.89E-32
Epoch 3/399 : Mon May  6 19:48:05 2019 - 0h 13m
----------

train Loss: 7.7901 Epoch Acc: 0.00E+00 Epoch Prob: 5.86E-09 Best Probability: 1.89E-32
val Loss: 6.7200 Epoch Acc: 0.00E+00 Epoch Prob: 1.10E-36 Best Probability: 1.89E-32
Epoch 4/399 : Mon May  6 19:51:58 2019 - 0h 17m
----------

train Loss: 6.0019 Epoch Acc: 0.00E+00 Epoch Prob: 7.53E-09 Best Probability: 1.89E-32
val Loss: 5.1697 Epoch Acc: 0.00E+0

Checkpoint saved
Epoch 35/399 : Mon May  6 21:52:28 2019 - 2h 17m
----------

train Loss: 1.9006 Epoch Acc: 4.00E-05 Epoch Prob: 4.65E-06 Best Probability: 3.48E-31
val Loss: 3.8689 Epoch Acc: 0.00E+00 Epoch Prob: 4.45E-35 Best Probability: 3.48E-31
Epoch 36/399 : Mon May  6 21:56:22 2019 - 2h 21m
----------

train Loss: 1.8864 Epoch Acc: 0.00E+00 Epoch Prob: 2.77E-06 Best Probability: 3.48E-31
val Loss: 3.6819 Epoch Acc: 0.00E+00 Epoch Prob: 1.57E-40 Best Probability: 3.48E-31
Epoch 37/399 : Mon May  6 22:00:16 2019 - 2h 25m
----------

train Loss: 1.8850 Epoch Acc: 0.00E+00 Epoch Prob: 6.30E-08 Best Probability: 3.48E-31
val Loss: 3.6611 Epoch Acc: 0.00E+00 Epoch Prob: 5.40E-38 Best Probability: 3.48E-31
Epoch 38/399 : Mon May  6 22:04:10 2019 - 2h 29m
----------

train Loss: 1.8495 Epoch Acc: 0.00E+00 Epoch Prob: 7.44E-07 Best Probability: 3.48E-31
val Loss: 3.5157 Epoch Acc: 0.00E+00 Epoch Prob: 8.04E-47 Best Probability: 3.48E-31
Epoch 39/399 : Mon May  6 22:08:03 2019 - 2h 33m
--

val Loss: 3.7520 Epoch Acc: 0.00E+00 Epoch Prob: 1.45E-38 Best Probability: 3.48E-31
Checkpoint saved
Epoch 70/399 : Tue May  7 00:08:53 2019 - 4h 34m
----------

train Loss: 1.5372 Epoch Acc: 0.00E+00 Epoch Prob: 3.78E-07 Best Probability: 3.48E-31
val Loss: 3.8075 Epoch Acc: 0.00E+00 Epoch Prob: 1.57E-44 Best Probability: 3.48E-31
Epoch 71/399 : Tue May  7 00:12:43 2019 - 4h 38m
----------

train Loss: 1.5748 Epoch Acc: 4.00E-05 Epoch Prob: 2.30E-06 Best Probability: 3.48E-31
val Loss: 3.7207 Epoch Acc: 0.00E+00 Epoch Prob: 2.11E-35 Best Probability: 3.48E-31
Epoch 72/399 : Tue May  7 00:16:34 2019 - 4h 42m
----------

train Loss: 1.5701 Epoch Acc: 0.00E+00 Epoch Prob: 1.23E-06 Best Probability: 3.48E-31
val Loss: 3.4998 Epoch Acc: 0.00E+00 Epoch Prob: 1.92E-48 Best Probability: 3.48E-31
Epoch 73/399 : Tue May  7 00:20:25 2019 - 4h 45m
----------

train Loss: 1.5485 Epoch Acc: 0.00E+00 Epoch Prob: 1.97E-07 Best Probability: 3.48E-31
val Loss: 3.5486 Epoch Acc: 0.00E+00 Epoch Prob: 8.

...

train Loss: 1.3239 Epoch Acc: 0.00E+00 Epoch Prob: 1.69E-07 Best Probability: 2.91E-24
val Loss: 3.7407 Epoch Acc: 0.00E+00 Epoch Prob: 6.63E-33 Best Probability: 2.91E-24
Epoch 242/399 : Tue May  7 11:09:49 2019 - 15h 35m
----------

train Loss: 1.3351 Epoch Acc: 0.00E+00 Epoch Prob: 1.08E-07 Best Probability: 2.91E-24
val Loss: 3.5611 Epoch Acc: 0.00E+00 Epoch Prob: 7.02E-38 Best Probability: 2.91E-24
Epoch 243/399 : Tue May  7 11:13:40 2019 - 15h 39m
----------

train Loss: 1.2864 Epoch Acc: 0.00E+00 Epoch Prob: 2.81E-06 Best Probability: 2.91E-24
val Loss: 3.5963 Epoch Acc: 0.00E+00 Epoch Prob: 1.12E-39 Best Probability: 2.91E-24
Epoch 244/399 : Tue May  7 11:17:31 2019 - 15h 43m
----------

train Loss: 1.2757 Epoch Acc: 0.00E+00 Epoch Prob: 1.10E-07 Best Probability: 2.91E-24
val Loss: 3.6986 Epoch Acc: 0.00E+00 Epoch Prob: 1.00E-37 Best Probability: 2.91E-24
Checkpoint saved
Epoch 245/399 : Tue May  7 11:21:22 2019 - 15h 46m
----------

train Loss: 1.2831 Epoch Acc: 0.00E+00 Epoc

val Loss: 3.5364 Epoch Acc: 0.00E+00 Epoch Prob: 2.22E-43 Best Probability: 2.91E-24
Epoch 276/399 : Tue May  7 13:20:36 2019 - 17h 46m
----------

train Loss: 1.2962 Epoch Acc: 0.00E+00 Epoch Prob: 5.31E-06 Best Probability: 2.91E-24
val Loss: 3.5066 Epoch Acc: 0.00E+00 Epoch Prob: 1.05E-46 Best Probability: 2.91E-24
Epoch 277/399 : Tue May  7 13:24:26 2019 - 17h 49m
----------

train Loss: 1.2745 Epoch Acc: 0.00E+00 Epoch Prob: 1.12E-09 Best Probability: 2.91E-24
val Loss: 3.7231 Epoch Acc: 0.00E+00 Epoch Prob: 4.85E-43 Best Probability: 2.91E-24
Epoch 278/399 : Tue May  7 13:28:16 2019 - 17h 53m
----------

train Loss: 1.2602 Epoch Acc: 0.00E+00 Epoch Prob: 3.40E-07 Best Probability: 2.91E-24
val Loss: 3.6299 Epoch Acc: 0.00E+00 Epoch Prob: 1.47E-39 Best Probability: 2.91E-24
Epoch 279/399 : Tue May  7 13:32:06 2019 - 17h 57m
----------

train Loss: 1.2800 Epoch Acc: 0.00E+00 Epoch Prob: 2.56E-06 Best Probability: 2.91E-24
val Loss: 3.6689 Epoch Acc: 0.00E+00 Epoch Prob: 3.79E-44 Be


train Loss: 1.2419 Epoch Acc: 0.00E+00 Epoch Prob: 2.50E-08 Best Probability: 2.91E-24
val Loss: 3.6918 Epoch Acc: 0.00E+00 Epoch Prob: 2.40E-40 Best Probability: 2.91E-24
Epoch 311/399 : Tue May  7 15:35:03 2019 - 20h 0m
----------

train Loss: 1.3038 Epoch Acc: 0.00E+00 Epoch Prob: 5.69E-07 Best Probability: 2.91E-24
val Loss: 3.7362 Epoch Acc: 0.00E+00 Epoch Prob: 3.42E-38 Best Probability: 2.91E-24
Epoch 312/399 : Tue May  7 15:38:54 2019 - 20h 4m
----------

train Loss: 1.2767 Epoch Acc: 0.00E+00 Epoch Prob: 1.05E-06 Best Probability: 2.91E-24
val Loss: 3.5832 Epoch Acc: 0.00E+00 Epoch Prob: 3.39E-39 Best Probability: 2.91E-24
Epoch 313/399 : Tue May  7 15:42:44 2019 - 20h 8m
----------

train Loss: 1.2474 Epoch Acc: 0.00E+00 Epoch Prob: 1.21E-06 Best Probability: 2.91E-24
val Loss: 3.6519 Epoch Acc: 0.00E+00 Epoch Prob: 6.05E-42 Best Probability: 2.91E-24
Epoch 314/399 : Tue May  7 15:46:35 2019 - 20h 12m
----------

train Loss: 1.2808 Epoch Acc: 0.00E+00 Epoch Prob: 1.60E-07 Be

val Loss: 3.6482 Epoch Acc: 0.00E+00 Epoch Prob: 1.73E-43 Best Probability: 2.91E-24
Checkpoint saved
Epoch 345/399 : Tue May  7 17:45:36 2019 - 22h 11m
----------

train Loss: 1.2310 Epoch Acc: 0.00E+00 Epoch Prob: 2.50E-07 Best Probability: 2.91E-24
val Loss: 3.6124 Epoch Acc: 0.00E+00 Epoch Prob: 1.45E-34 Best Probability: 2.91E-24
Epoch 346/399 : Tue May  7 17:49:26 2019 - 22h 14m
----------

train Loss: 1.3215 Epoch Acc: 0.00E+00 Epoch Prob: 3.58E-07 Best Probability: 2.91E-24
val Loss: 3.6416 Epoch Acc: 0.00E+00 Epoch Prob: 2.59E-44 Best Probability: 2.91E-24
Epoch 347/399 : Tue May  7 17:53:16 2019 - 22h 18m
----------

train Loss: 1.2985 Epoch Acc: 0.00E+00 Epoch Prob: 5.69E-07 Best Probability: 2.91E-24
val Loss: 3.6681 Epoch Acc: 0.00E+00 Epoch Prob: 2.06E-36 Best Probability: 2.91E-24
Epoch 348/399 : Tue May  7 17:57:07 2019 - 22h 22m
----------

train Loss: 1.2328 Epoch Acc: 0.00E+00 Epoch Prob: 1.03E-06 Best Probability: 2.91E-24
val Loss: 3.6254 Epoch Acc: 0.00E+00 Epoch 


train Loss: 1.2701 Epoch Acc: 0.00E+00 Epoch Prob: 3.10E-06 Best Probability: 2.91E-24
val Loss: 3.8702 Epoch Acc: 0.00E+00 Epoch Prob: 2.38E-29 Best Probability: 2.91E-24
Checkpoint saved
Epoch 380/399 : Tue May  7 20:02:11 2019 - 24h 27m
----------

train Loss: 1.2179 Epoch Acc: 0.00E+00 Epoch Prob: 4.91E-06 Best Probability: 2.91E-24
val Loss: 3.6270 Epoch Acc: 0.00E+00 Epoch Prob: 9.79E-40 Best Probability: 2.91E-24
Epoch 381/399 : Tue May  7 20:06:25 2019 - 24h 31m
----------

train Loss: 1.2851 Epoch Acc: 0.00E+00 Epoch Prob: 3.74E-07 Best Probability: 2.91E-24
val Loss: 3.6208 Epoch Acc: 0.00E+00 Epoch Prob: 4.36E-41 Best Probability: 2.91E-24
Epoch 382/399 : Tue May  7 20:10:40 2019 - 24h 36m
----------

train Loss: 1.2382 Epoch Acc: 0.00E+00 Epoch Prob: 8.66E-08 Best Probability: 2.91E-24
val Loss: 3.5670 Epoch Acc: 0.00E+00 Epoch Prob: 5.95E-40 Best Probability: 2.91E-24
Epoch 383/399 : Tue May  7 20:14:50 2019 - 24h 40m
----------

train Loss: 1.3275 Epoch Acc: 0.00E+00 Epo

In [34]:
out = model_ft(test_batch)
out = out.squeeze(3)
out2 = out.permute(0,2,1)
outText = []
loader = DataLoader(filePath = FilePaths.fnTrain, batchSize = Model.batchSize, imgSize = Model.imgSize, maxTextLen = Model.maxTextLen)
charList = loader.charList
for i in range (out2.shape[0]):
    softmaxOut = torch.nn.functional.softmax(out2[i], dim = 1).cuda()
    softmaxOut = softmaxOut.cpu().detach().numpy()
    print (softmaxOut[0][0])
    outText.append(BestPath.ctcBestPath(softmaxOut, charList))
print (outText)
print (test_labels)

1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
1.0
0.99988544
0.99990237
1.0
0.9999596
0.9951139
1.0
1.0
0.9999596
0.9994369
0.94125503
0.9999747
0.9999664
0.9999747
1.0
0.9999838
0.9999734
0.99997485
1.0
0.99997306
1.0
1.0
0.9834812
0.99998367
0.9998103
1.0
0.99986017
0.9999713
0.99997175
0.9999732
1.0
0.98067427
0.99993765
1.0
0.9996618
0.99883
0.99997187
0.9998204
0.9456496
1.0
0.9664101
0.99999464
1.0
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['bit', ',', 'Di', ',', '"', 'he', 'told', 'her', '.', '"', 'But', 'I', 'want', 'it', '!', '"', 'she', 'protested', '.', '"', 'It', "'s", 'my', 'ring', '.', '"', '"', 'You', 'just', 'gave', 'it', 'back', 'to', 'me', '.', '"', '"', 'I', "didn't", '!', 'I', 'was', 'joking', '.', 'You', 'know', 'I', 'was', 'joking', ',']


In [30]:
out = model_ft(test_batch)
out = out.squeeze(3)
out2 = out.permute(0,2,1)
#rnn = rnn.squeeze(3)
print(rnn.size())
print (test_batch.shape[0])

torch.Size([50, 512])
torch.Size([50, 512])
50


In [33]:

cv2.imshow('original vs sharpened image', test_batch[2])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
for i in range (rnn.shape[0]):
    print(rnn[i][4])

tensor(0.0109, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0113, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0113, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0128, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0107, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0111, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0111, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0109, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0120, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0109, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0154, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0120, device='cuda:0', grad_fn=<SelectBackward>)
tensor(-0.0216, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0157, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0121, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0120, device='cuda:0', grad_fn=<SelectBackward>)
tensor(0.0112, device='cuda:0', grad_fn=<SelectBackward>)
tensor(-0.057